<h4> Web Scrape Module </h4>

The purpose of this module is to bypass complicated astronomy math using number that would likely need to be scraped anyway such as hour angle and latitude.
<br>
<br>
<i>
Elevation angle:
<br>
</i>
The elevation angle is the angular height of the sun in the sky measured from the horizontal.

α=sin−1 (sinδ sinϕ +cosδ cosϕ cos(HRA))

Where δ is the declination angle, ϕ is the local latitude and HRA is the Hour angle.

Zenith Angle:

The zenith angle is the angle between the sun and the vertical. The zenith angle is similar to the elevation angle but it is measured from the vertical rather than from the horizontal, thus making the zenith
Zenith=90°−α

Where α is the elevation angle.

<br>
By calling solartopo.com it is possible to fill out the less common info our model uses to predict a target W/m^2

In [132]:
from time import sleep, strftime
from random import randint
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup


#imports for SMTP
import smtplib
from os.path import basename
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

Me = "Users/Mark/Downloads"

chromedriver_path = f'C:/{Me}/chromedriver_win32/chromedriver.exe'

chromedriver_path

'C:/Users/Mark/Downloads/chromedriver_win32/chromedriver.exe'

In [50]:
driver = webdriver.Chrome(executable_path=chromedriver_path)
sleep(.25)

In [76]:
# update latitude and longitude to 
location = 80232
# lat = # call to where lat is stored
# long = # call to where long is stored
current_time = datetime.datetime.now()
# Get's yesterday, the most current full day of data
default_time = (f'{current_time.day-1}/{current_time.month}/{current_time.year}.{current_time.hour}:{current_time.minute}') # time must be in this format

In [ ]:
# sun_calc = (f'https://www.suncalc.org/#/{lat},{long},9/{time}/324.0/2')

In [19]:
solartopo = 'http://solartopo.com'
orbit = '/solar-orbit.htm'
daylight = '/daylength.htm'

In [20]:
#  SunCalc

# //*[@id="sunhoehe"] #
# //*[@id="clickSunrise"] # sunrise (listens for click)

In [21]:
#  Solar Topo

date_picker = '//*[@id="datepicker"]'
address_bar = '//*[@id="address"]'
submit = '/html/body/div[1]/div[9]/div[1]/input[2]'

<h4> Sunpath Hit </h4>

In [72]:
def get_sunpath():
    '''
    helper function that retrieves the sunpath data.
    '''
    zenith = driver.find_element_by_xpath('//*[@id="zenith"]').text
    azimuth = driver.find_element_by_xpath('//*[@id="azimuth"]').text
    if zenith == '' or azimuth == '':
        driver.refresh();
    zenith = float(zenith.split('°')[0])
    azimuth = float(azimuth.split('°')[0])

    return azimuth, zenith

<h4> Sunrise & Sunset Hit </h4>

In [61]:
def get_time(user_input = None):
    '''
    gets time in the right formats for our calls
    outputs are in strings
    user_input takes a user input rather than the current time
    
    '''
    if user_input:
#         will be a sol format datefield string  mm/dd/yyyy
        units = user_input.split('/')
    else:
        current_time = datetime.datetime.now() #this doesn't run?
        units = [current_time.year, current_time.month, 
                 current_time.day, current_time.hour, 
                 current_time.minute]
    time_list = []
    for unit in units:
        if unit < 10:
            unit = '0' + str(unit)
        time_list.append(unit)
    return time_list

def convert_time(time_list, sol = True):
    '''
    turns the time_list into the correct format for specific calls
    if sol, format for solortopo.  if not, format for darksky
    returns a string
    '''
    time = (f'{time_list[3]}:{time_list[4]}:00')
    if sol:
        solartopo_date = (f'{time_list[2]-1}/{time_list[1]}/{time_list[0]}')
        return solartopo_date, time
    else:
        darksky_date = (f'{time_list[0]}-{time_list[1]}-{time_list[2]-1}')
        return darksky_date, time

In [126]:
time_list = get_time()
sol, time = convert_time(time_list)
sol

'24/08/2019'

In [123]:
def get_next_day(dates):
    '''
    time is relative, add a day.
    accepts sol format
    '''
    split = dates.split('/')
    integers = [int(x) for x in split]
    integers[0] += 1
    string = '/'.join(str(num) for num in integers)
    return string

In [122]:
sol2 = get_next_day(sol)


['24', '08', '2019']
[25, 8, 2019]
25/8/2019


In [129]:
sunrise = 600
sunset = 1000

def collect_data(time_span=2, location=80232, date = sol, sunrise = sunrise, sunset = sunset):
    '''
    function takes a time in days, a zipcode, and a date
    and prepares a dictionary of information for as many days as time_span designates
    '''
    
    sun_getter = (f'{solartopo}{orbit}')
    driver.get(sun_getter)
    sleep(randint(2,3))
    
    
    for i in range(time_span):
    
        driver.find_element_by_xpath(f"{date_picker}").clear()
        driver.find_element_by_xpath(f"{date_picker}").send_keys(f'{date}')
        driver.find_element_by_xpath(f"{address_bar}").clear()
        driver.find_element_by_xpath(f"{address_bar}").send_keys(f'{location}')
        driver.find_element_by_xpath(f"{submit}").click()
    
    # scrape sunrise sunset(from previous data)
# 
#     
        
        # Trun the clock back in order to get chronologic readings

        display_time = driver.find_element_by_xpath('//*[@id="time"]').text
        slider = driver.find_element_by_xpath('//*[@id="slider-range-min"]/a')
        while convert_time_minutes(display_time) > sunrise:
            print(display_time)
            display_time = driver.find_element_by_xpath('//*[@id="time"]').text
            slider.send_keys(Keys.ARROW_LEFT + Keys.ARROW_LEFT + Keys.ARROW_LEFT + Keys.ARROW_LEFT + Keys.ARROW_LEFT
                            + Keys.ARROW_LEFT + Keys.ARROW_LEFT + Keys.ARROW_LEFT + Keys.ARROW_LEFT + Keys.ARROW_LEFT)
            
        # Change the date in preperation for the next call
        
        sol = get_next_day(date)
        
        # Collect readings for daylight hours
        
        for reading in range(time_span):
            slider.click()
            # hard coded for 15 minute intervals 24 * 4 = 96 are the max intervals in one day
            for i in range(96):
                slider.send_keys(Keys.ARROW_RIGHT + Keys.ARROW_RIGHT + Keys.ARROW_RIGHT + Keys.ARROW_RIGHT + Keys.ARROW_RIGHT
                            + Keys.ARROW_RIGHT + Keys.ARROW_RIGHT + Keys.ARROW_RIGHT + Keys.ARROW_RIGHT + Keys.ARROW_RIGHT
                                + Keys.ARROW_RIGHT + Keys.ARROW_RIGHT + Keys.ARROW_RIGHT + Keys.ARROW_RIGHT + Keys.ARROW_RIGHT)
                minute = convert_time_minutes(driver.find_element_by_xpath('//*[@id="time"]').text)
                print(minute)
                if minute < sunrise:
                    pass
                elif minute >= sunset:
                    break
                else:
                    # scrape zenith / azimuth
                    zenith, azimuth = get_sunpath()
                    # append zenith / azimuth to 
                
    return

In [130]:
collect_data()

12:00
12:00
11:50
11:40
11:30
11:20
11:10
11:00
10:50
10:40
10:30
10:20
10:10
605
41.45° 113.7°
620
44.03° 117.29°
635
46.56° 121.09°
650
48.95° 125.29°
665
51.25° 129.75°
680
53.35° 134.73°
695
55.32° 140.03°
710
57.02° 145.95°
725
58.53° 152.19°
740
59.67° 159.06°
755
60.58° 166.15°
770
61.02° 173.68°
785
61.21° 181.22°
800
60.89° 188.8°
815
60.33° 196.18°
830
59.29° 203.19°
845
58.06° 209.89°
860
56.45° 215.97°
875
54.68° 221.73°
890
52.63° 226.86°
905
50.48° 231.72°
920
48.13° 236.03°
935
45.7° 240.13°
950
43.14° 243.81°
965
40.53° 247.33°
980
37.83° 250.53°
995
35.09° 253.63°
1010
1025
17:05
17:05
16:55
16:45
16:35
16:25
16:15
16:05
15:55
15:45
15:35
15:25
15:15
15:05
14:55
14:45
14:35
14:25
14:15
14:05
13:55
13:45
13:35
13:25
13:15
13:05
12:55
12:45
12:35
12:25
12:15
12:05
11:55
11:45
11:35
11:25
11:15
11:05
10:55
10:45
10:35
10:25
10:15
10:05
600
40.58° 112.5°
615
43.17° 116.1°
630
45.76° 119.69°
645
48.15° 123.89°
660
50.55° 128.08°
675
52.65° 133.07°
690
54.76° 138.05°
705
56.